# Day 3: Disney Parks Guest Spending Behavior

You are a data analyst working with the Disney Parks revenue team to understand nuanced guest spending patterns across different park experiences. The team wants to develop a comprehensive view of visitor purchasing behaviors. Your goal is to uncover meaningful insights that can drive personalized marketing strategies.

In [ ]:
import pandas as pd
import numpy as np

fct_guest_spending_data = [
  {
    "guest_id": 1,
    "visit_date": "2024-07-05",
    "amount_spent": 50,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-07-06",
    "amount_spent": 30,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-07-10",
    "amount_spent": 20.5,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 4,
    "visit_date": "2024-07-12",
    "amount_spent": 40,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-07-15",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 5,
    "visit_date": "2024-07-20",
    "amount_spent": 60,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 6,
    "visit_date": "2024-07-25",
    "amount_spent": 25,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-03",
    "amount_spent": 55,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-15",
    "amount_spent": 45,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-05",
    "amount_spent": 22,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-20",
    "amount_spent": 38,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 7,
    "visit_date": "2024-08-10",
    "amount_spent": 15,
    "park_experience_type": "Character Meet"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-25",
    "amount_spent": 28,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-27",
    "amount_spent": 32,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-02",
    "amount_spent": 65,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-05",
    "amount_spent": 50,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 9,
    "visit_date": "2024-09-15",
    "amount_spent": 40,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 10,
    "visit_date": "2024-09-20",
    "amount_spent": 70,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-25",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-28",
    "amount_spent": 10,
    "park_experience_type": "Character Meet"
  }
]
fct_guest_spending = pd.DataFrame(fct_guest_spending_data)


## Question 1

What is the average spending per guest per visit for each park experience type during July 2024? Ensure that park experience types with no recorded transactions are shown with an average spending of 0.0. This analysis helps establish baseline spending differences essential for later segmentation.

In [ ]:
# Note: pandas and numpy are already imported as pd and np
# The following tables are loaded as pandas DataFrames with the same names: fct_guest_spending
# Please print your final result or dataframe
fct_guest_spending['visit_date'] = pd.to_datetime(fct_guest_spending['visit_date'], errors = 'coerce')
fct_guest_spending_filtered = fct_guest_spending[(fct_guest_spending['visit_date'] >= '2024-07-01') & (fct_guest_spending['visit_date'] <= '2024-07-31')]

fct_guest_spending_grp = fct_guest_spending_filtered.groupby(['guest_id','visit_date','park_experience_type'], as_index = False)['amount_spent'].sum()

type_avg = (fct_guest_spending_grp.groupby('park_experience_type', as_index=False)['amount_spent'].mean().rename(columns={'amount_spent': 'avg_spending_per_guest_per_visit'}))

all_types = fct_guest_spending['park_experience_type'].dropna().drop_duplicates().to_frame(name='park_experience_type') 
            
type_avg_full = (all_types.merge(type_avg, on='park_experience_type', how='left').fillna({'avg_spending_per_guest_per_visit': 0.0}))

print(type_avg_full)

## Question 2

For guests who visited our parks more than once in August 2024, what is the difference in spending between their first and their last visit? This investigation, using sequential analysis, will reveal any shifts in guest spending behavior over multiple visits.

In [ ]:
fct_guest_spending['visit_date'] = pd.to_datetime(fct_guest_spending['visit_date'], errors = 'coerce')
fct_guest_spending_filtered = fct_guest_spending[(fct_guest_spending['visit_date'] >= '2024-08-01') & (fct_guest_spending['visit_date'] < '2024-09-01')]

fct_guest_spending_grp = fct_guest_spending_filtered.groupby(['guest_id','visit_date'], as_index = False)['amount_spent'].sum()

visits_grp = fct_guest_spending_grp.groupby('guest_id', as_index = False)['visit_date'].nunique().rename(columns = {'visit_date':'visit_count'})

visits_grp_filtered = visits_grp[(visits_grp['visit_count'] >= 2)]

guests_filtered = fct_guest_spending_grp.merge(visits_grp_filtered, on = 'guest_id')

idx_first = guests_filtered.groupby(['guest_id'])['visit_date'].idxmin() #is a series with selected rows of guest_id and row index where visit date is min
first_visit = (guests_filtered.loc[idx_first, ['guest_id','amount_spent']].rename(columns={'amount_spent':'amount_spent_first_visit'}))

idx_last = guests_filtered.groupby(['guest_id'])['visit_date'].idxmax() #is a series with selected rows of guest_id and row index where visit date is max
last_visit = (guests_filtered.loc[idx_last, ['guest_id','amount_spent']].rename(columns={'amount_spent':'amount_spent_last_visit'}))

visit_merged = first_visit.merge(last_visit, on = ['guest_id'])

visit_merged['diff'] = visit_merged['amount_spent_first_visit'] - visit_merged['amount_spent_last_visit']
print(visit_merged)

## Question 3

In September 2024, how can guests be categorized into distinct spending segments such as Low, Medium, and High based on their total spending? Use the following thresholds for categorization: 
-Low: Includes values from $0 up to, but not including, $50.
-Medium: Includes values from $50 up to, but not including, $100.
-High: Includes values from $100 and above. 
Exclude guests who did not make any purchases in the period.

In [ ]:
fct_guest_spending['visit_date'] = pd.to_datetime(fct_guest_spending['visit_date'], errors = 'coerce')
fct_guest_spending_filtered = fct_guest_spending[(fct_guest_spending['visit_date'] >= '2024-09-01') & (fct_guest_spending['visit_date'] < '2024-10-01')]
fct_guest_spending_grp = fct_guest_spending_filtered.groupby('guest_id', as_index = False)['amount_spent'].sum()
nonzero = fct_guest_spending_grp[fct_guest_spending_grp['amount_spent'] > 0]

bins = [0, 50, 100, float('inf')]
labels = ['Low', 'Medium', 'High']

nonzero['spend_segment'] = pd.cut(nonzero['amount_spent'], bins=bins, labels=labels, right=False)  # makes intervals [a, b)

print(nonzero)

Made with ❤️ by [Interview Master](https://www.interviewmaster.ai)